# 프로젝트: 단어 level로 번역기 업그레이드하기

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Embedding, Masking,LSTM,Dense
from tensorflow.keras.models import Model

import pandas as pd
import numpy as np
import os
import re

In [2]:
# tf.debugging.set_log_device_placement(True)

In [3]:
file_path = './data/fra.txt'
original_lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(original_lines))
original_lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 178009


,eng,fra,cc
150453,They supported his right to speak freely.,Elles soutinrent son droit à parler librement.,CC-BY 2.0 (France) Attribution: tatoeba.org #8...
97028,He wanted it to be a surprise.,Il voulut que ce fut une surprise.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
97836,I fully support your proposal.,Je soutiens intégralement ta proposition.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
114305,"You are a good cook, aren't you?","Tu es bon cuisinier, non ?",CC-BY 2.0 (France) Attribution: tatoeba.org #3...
68671,I can deliver that to Tom.,Je peux livrer cela à Tom.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


## 정제, 정규화, 전처리 (영어,프랑스어 모두!)

### 33000개만 사용

In [4]:
num_samples = 33000
lines = original_lines[['eng', 'fra']][:num_samples] # 3.3만개 샘플 사용
lines.sample(5)

,eng,fra
29698,I hate taking risks.,Je déteste prendre des risques.
17361,What shall we do?,Que faire ?
14060,Everyone laughed.,Tout le monde a ri.
20562,My brother is out.,Mon frère est dehors.
16046,No one ever came.,Personne ne vint jamais.



### 소문자로 변경

In [5]:
# lines['eng_separate']
lines['eng'] = lines['eng'].str.lower()
lines['fra'] = lines['fra'].str.lower()
# lines['eng_separate'].str.toLowerCase()

In [6]:
lines['eng']

0                         go.
1                         hi.
2                         hi.
3                        run!
4                        run!
                 ...         
32995    what was their goal?
32996    what were you doing?
32997    what would tom need?
32998    what would you like?
32999    what would you like?
Name: eng, Length: 33000, dtype: object

In [7]:
lines['fra']

0                                   va !
1                                salut !
2                                 salut.
3                                cours !
4                               courez !
                      ...               
32995              quel était leur but ?
32996    qu'étais-tu en train de faire ?
32997     de quoi tom aurait-il besoin ?
32998                   qu'aimerais-tu ?
32999                 qu'aimeriez-vous ?
Name: fra, Length: 33000, dtype: object

### 띄어쓰기 단위로 구분 && 구두점을 단어와 분리

In [8]:
import re
# import string
# string.punctuation

lines['eng_separate'] = lines['eng'].str.findall(r"[\w']+|[.,!?;]")
lines['fra_separate'] = lines['fra'].str.findall(r"[\w']+|[.,!?;]")

In [9]:
lines['eng_separate']

0                            [go, .]
1                            [hi, .]
2                            [hi, .]
3                           [run, !]
4                           [run, !]
                    ...             
32995    [what, was, their, goal, ?]
32996    [what, were, you, doing, ?]
32997    [what, would, tom, need, ?]
32998    [what, would, you, like, ?]
32999    [what, would, you, like, ?]
Name: eng_separate, Length: 33000, dtype: object

In [10]:
lines['fra_separate']

0                                        [va, !]
1                                     [salut, !]
2                                     [salut, .]
3                                     [cours, !]
4                                    [courez, !]
                          ...                   
32995                [quel, était, leur, but, ?]
32996    [qu'étais, tu, en, train, de, faire, ?]
32997     [de, quoi, tom, aurait, il, besoin, ?]
32998                       [qu'aimerais, tu, ?]
32999                     [qu'aimeriez, vous, ?]
Name: fra_separate, Length: 33000, dtype: object

## 시작, 종료 토큰 추가

In [11]:
# 시작 토큰과 종료 토큰 추가
sos_token = '\t'
eos_token = '\n'
# lines.fra = lines.fra.apply(lambda x : '\t '+ x + ' \n')
# print('전체 샘플의 수 :',len(lines))
# lines.sample(5)

lines['fra_separate_token'] = lines.fra_separate.apply(
    lambda x : ['\t'] + x + ['\n']
    )


In [12]:
lines.sample(5)

,eng,fra,eng_separate,fra_separate,fra_separate_token
14596,i bowed politely.,je m'inclinai poliment.,"[i, bowed, politely, .]","[je, m'inclinai, poliment, .]","[\t, je, m'inclinai, poliment, ., \n]"
1251,here we go.,on est partis !,"[here, we, go, .]","[on, est, partis, !]","[\t, on, est, partis, !, \n]"
5296,he's ruthless.,il est sans pitié.,"[he's, ruthless, .]","[il, est, sans, pitié, .]","[\t, il, est, sans, pitié, ., \n]"
20510,"long time, no see.",ça fait une paille.,"[long, time, ,, no, see, .]","[ça, fait, une, paille, .]","[\t, ça, fait, une, paille, ., \n]"
18873,how can i do that?,comment puis-je faire cela ?,"[how, can, i, do, that, ?]","[comment, puis, je, faire, cela, ?]","[\t, comment, puis, je, faire, cela, ?, \n]"


## 케라스 토크나이저로 텍스트를 숫자로 변경

In [13]:
# tf.keras.preprocessing.text.Tokenizer(
#     num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,
#     split=' ', char_level=False, oov_token=None, document_count=0, **kwargs
# )

eng_tokenizer = Tokenizer(char_level=False)   # 단어 단위로 Tokenizer를 생성합니다. 
eng_tokenizer.fit_on_texts(lines.eng_separate)               # 33000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(lines.eng_separate)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[27, 1], [1160, 1], [1160, 1]]

In [14]:
fra_tokenizer = Tokenizer(char_level=False)   # 문자 단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(lines.fra_separate_token)                 # 50000개의 행을 가진 fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra_separate_token)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[1, 82, 10, 2], [1, 1084, 10, 2], [1, 1084, 3, 2]]

In [15]:
fra_tokenizer.index_word[1]

'\t'

### 단어장 크기 저장

In [16]:
# 0번 토큰을 고려한 +1
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 4796
프랑스어 단어장의 크기 : 9034


In [17]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 15


In [18]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 4796
프랑스어 단어장의 크기 : 9034
영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 15


###  프랑스어 시퀀스 2개 버전 준비
- 하나는 디코더의 출력과 비교해야 할 **정답 데이터**로 사용해야 할 원래 목적에 따른 것입니다.
- 하나는 이전 스텝에서 언급했던 **교사 강요(Teacher forcing)을 위해** **디코더의 입력**으로 사용하기 위한 것입니다.

In [19]:
# endocer input은 input_text 그대로
encoder_input = input_text

# 종료 토큰 제거 ( 교사 강요를 위해 decoder input으로 들어가는거니까 )
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거 ( decoder 출력과 비교하는 값이니까 )
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [20]:
# from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 8)
프랑스어 입력데이터의 크기(shape) : (33000, 15)
프랑스어 출력데이터의 크기(shape) : (33000, 15)


In [21]:
print(encoder_input[0])

[27  1  0  0  0  0  0  0]


### 원 핫 인코딩
단어 단위로 하니까 안한다.

In [22]:
# encoder_input = to_categorical(encoder_input)
# decoder_input = to_categorical(decoder_input)
# decoder_target = to_categorical(decoder_target)
# print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
# print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
# print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

### 데이터 쪼개기

In [23]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input_train))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input_train))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target_train))

영어 학습데이터의 크기(shape) : (30000, 8)
프랑스어 학습 입력데이터의 크기(shape) : (30000, 15)
프랑스어 학습 출력데이터의 크기(shape) : (30000, 15)


## 임베딩 층(Embedding layer) 사용하기
https://wikidocs.net/33793

어떤 단어 → 단어에 부여된 고유한 정수값 → 임베딩 층 통과 → 밀집 벡터   

임베딩 층은 입력 정수에 대해 밀집 벡터(dense vector)로 맵핑하고 이 밀집 벡터는 인공 신경망의 학습 과정에서 가중치가 학습되는 것과 같은 방식으로 훈련됩니다. 훈련 과정에서 단어는 모델이 풀고자하는 작업에 맞는 값으로 업데이트 됩니다. 그리고 이 밀집 벡터를 임베딩 벡터라고 부릅니다.


인코더와 디코더의 임베딩 층은 서로 다른 임베딩 층을 사용해야 하지만,   
디코더의 훈련 과정과 테스트 과정(예측 과정)에서의 임베딩 층은 동일해야 합니다.

In [24]:
# 인코더에서 사용할 임베딩 층 사용 예시
# 입력 텐서 생성 (입력 문장을 저장하게 될 변수 텐서)
encoder_inputs = Input(shape=(None,))

enc_emb_dim = 256
enc_emb =  Embedding(eng_vocab_size, enc_emb_dim,input_length=max_eng_seq_len)(encoder_inputs)

enc_masking = Masking(mask_value=0.0)(enc_emb)

# hidden size는 LSTM의 수용력을 의미한다.
encoder_lstm = LSTM(units = 256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

# # 입력 텐서 생성.
# encoder_inputs = Input(shape=(None, eng_vocab_size))
# # hidden size가 256인 인코더의 LSTM 셀 생성
# encoder_lstm = LSTM(units = 256, return_state = True)
# # 디코더로 전달할 hidden state, cell state를 리턴. encoder_outputs은 여기서는 불필요.
# encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# # hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.
# encoder_states = [state_h, state_c]

In [25]:
encoder_states

[<tf.Tensor 'lstm/PartitionedCall:2' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'lstm/PartitionedCall:3' shape=(None, 256) dtype=float32>]

In [26]:
# 입력 텐서 생성
decoder_inputs = Input(shape=(None,))

dec_emb_dim = 256
dec_emb =  Embedding(fra_vocab_size, dec_emb_dim)(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)

decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state = encoder_states)

decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)
# # 입력 텐서 생성.
# decoder_inputs = Input(shape=(None, fra_vocab_size))
# # hidden size가 256인 인코더의 LSTM 셀 생성
# decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
# # decoder_outputs는 모든 time step의 hidden state
# decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state = encoder_states)

In [27]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 256)    1227776     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    2312704     input_2[0][0]                    
_______________________________________________________________________________________

In [28]:
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy",  metrics=['acc'])

In [29]:
# tf.debugging.set_log_device_placement(True)

In [30]:
# # model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
# #           validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
# #           batch_size=128, epochs=50, verbose = 2)

# 메모리 터져서 batch_size 32
model.fit([encoder_input_train, decoder_input_train],decoder_target_train,
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 32, epochs = 50)

Epoch 1/50
938/938 [==============================] - 20s 21ms/step - loss: 1.6382 - acc: 0.7522 - val_loss: 1.6675 - val_acc: 0.7548
Epoch 2/50
938/938 [==============================] - 18s 20ms/step - loss: 1.2291 - acc: 0.8112 - val_loss: 1.4896 - val_acc: 0.7805
Epoch 3/50
938/938 [==============================] - 18s 20ms/step - loss: 1.1031 - acc: 0.8283 - val_loss: 1.4009 - val_acc: 0.7906
Epoch 4/50
938/938 [==============================] - 18s 20ms/step - loss: 1.0154 - acc: 0.8403 - val_loss: 1.3486 - val_acc: 0.7980
Epoch 5/50
938/938 [==============================] - 18s 20ms/step - loss: 0.9505 - acc: 0.8493 - val_loss: 1.3157 - val_acc: 0.8026
Epoch 6/50
938/938 [==============================] - 18s 20ms/step - loss: 0.8977 - acc: 0.8585 - val_loss: 1.2895 - val_acc: 0.8096
Epoch 7/50
938/938 [==============================] - 19s 20ms/step - loss: 0.8572 - acc: 0.8659 - val_loss: 1.2812 - val_acc: 0.8104
Epoch 8/50
938/938 [==============================] - 19s 20ms

## 모델 구현(테스트)

### 인코더 설계

In [31]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 256)         1227776   
_________________________________________________________________
masking (Masking)            (None, None, 256)         0         
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 525312    
Total params: 1,753,088
Trainable params: 1,753,088
Non-trainable params: 0
_________________________________________________________________


### 디코더 설계

In [32]:
# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(256,))
# # 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_states_inputs

[<tf.Tensor 'input_3:0' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'input_4:0' shape=(None, 256) dtype=float32>]

In [33]:
dec_emb_test = Embedding(fra_vocab_size, dec_emb_dim)(decoder_inputs)
decoder_outputs_test, state_h_test, state_c_test = decoder_lstm(dec_emb_test, initial_state = decoder_states_inputs)

decoder_states_test = [state_h_test, state_c_test]

decoder_outputs_test = decoder_softmax_layer(decoder_outputs_test)

In [34]:
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs_test] + decoder_states_test)
decoder_model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 256)    2312704     input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 256)]        0                                            
_______________________________________________________________________________________

In [35]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

# print(idx2fra)

In [62]:
fra2idx['\t']

1

### 예측 과정을 위한 decode_sequence() 함수 구현

decode_sequence() 내부에는 인코더를 구현한 encoder_model이 있어서 이 모델에 번역하고자 하는 문장의 정수 시퀀스인 'input_seq'를 입력하면, encoder_model은 마지막 시점의 hidden state를 리턴합니다.

이 hidden state는 디코더의 첫번째 시점의 hidden state가 되고, 디코더는 이제 번역 문장을 완성하기 위한 예측 과정을 진행합니다. 디코더의 예측 과정에서는 이전 시점에서 예측한 단어를 디코더의 현재 시점의 입력으로 넣어주는 작업을 진행합니다. 그리고 이 작업은 종료를 의미하는 종료 토큰을 만나거나, 주어진 최대 길이를 넘을 때까지 반복합니다.

In [68]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = fra2idx['\t']

    stop_condition = False
    decoded_sentence = ""

    # print(states_value[0])
    # print(target_seq)

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # print(output_tokens)
        # print(output_tokens.shape)
        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        # print(sampled_token_index)
        # sampled_token_index+=1
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가 ( 띄어쓰기좀 해주자)
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [69]:
import numpy as np
for seq_index in [4,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input_test[seq_index: seq_index + 1]
    # print(input_seq.shape)
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][1:len(decoder_input_test[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장 길이 :',len(decoded_sentence))
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: run!
정답 문장: ourez !
번역기가 번역한 문장 길이 : 16
번역기가 번역한 문장:  je je à à à à 
-----------------------------------
입력 문장: i left.
정답 문장: e suis partie
번역기가 번역한 문장 길이 : 19
번역기가 번역한 문장:  je me . . . parti
-----------------------------------
입력 문장: call us.
정답 문장: ppelez-nous !
번역기가 번역한 문장 길이 : 17
번역기가 번역한 문장:  je . . . . . . 
-----------------------------------
입력 문장: how nice!
정답 문장: omme c'est ge
번역기가 번역한 문장 길이 : 16
번역기가 번역한 문장:  j'étais été ét
-----------------------------------
입력 문장: turn left.
정답 문장: ourne à gauch
번역기가 번역한 문장 길이 : 18
번역기가 번역한 문장:  c'est vraiment t


# 결과
프랑스어를 할 줄 모르는 내가 보기에도 굉장히 말도 안되는 결과물들이 나오고 있다.
decoder 모델 설계에 문제가 있는 것으로 생각된다.